In [ ]:
%matplotlib nbagg
import matplotlib.pyplot as plt
from IPython.display import display
##
import json
import re
import numpy           as np
import pandas          as pd
import statsmodels.api as sm

##
import thundermint.logs  as Log
import thundermint.plot  as plot
import thundermint.splot as splot

In [ ]:
logs = Log.load_logs_files('../thundermint/logs/', ['node-1.log','node-2.log','node-3.log','node-4.log'])
p = plot.CommitData(logs)

# Plots

In [ ]:
with plot.LegendPlot() as ax:
    p.plot_points(ax, reltime=True)
with plot.LegendPlot() as ax:
    p.plot_residuals_TvsH(ax)
plot.plot_round(logs)
None

In [ ]:
with plot.SimplePlot() as ax:
    p.plot_ntx(ax)
with plot.SimplePlot() as ax:
    p.plot_ntx_distr(ax)
with plot.SimplePlot() as ax:
    p.plot_n_signatures(ax)

In [ ]:
plot.plot_mempool_size(logs)
None

In [ ]:
plot.plot_mempool_added(logs)
None

In [ ]:
splot.splot(logs, w=8096)